In [0]:
from pyspark.sql.functions import col, desc
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

Connect databricks to datalake

In [1]:
# insert configs

In [0]:
%fs
ls "mnt/olympicdata"

path,name,size,modificationTime
dbfs:/mnt/olympicdata/raw-data/,raw-data/,0,1731664582000
dbfs:/mnt/olympicdata/transformed-data/,transformed-data/,0,1731664591000


In [0]:
spark

load data in spark

In [0]:
athletes = spark.read.format("csv").option("header", True).option("inferSchema", True).load("/mnt/olympicdata/raw-data/athletes.csv")
coaches = spark.read.format("csv").option("header", True).option("inferSchema", True).load("/mnt/olympicdata/raw-data/coaches.csv")
entriesgender = spark.read.format("csv").option("header", True).option("inferSchema", True).load("/mnt/olympicdata/raw-data/entries-gender.csv")
medals = spark.read.format("csv").option("header", True).option("inferSchema", True).load("/mnt/olympicdata/raw-data/medals.csv")
teams = spark.read.format("csv").option("header", True).option("inferSchema", True).load("/mnt/olympicdata/raw-data/teams.csv")

Explore Data

In [0]:
athletes.show(5)

+-----------------+------+-------------------+
|             Name|   NOC|         Discipline|
+-----------------+------+-------------------+
|  AALERUD Katrine|Norway|       Cycling Road|
|      ABAD Nestor| Spain|Artistic Gymnastics|
|ABAGNALE Giovanni| Italy|             Rowing|
|   ABALDE Alberto| Spain|         Basketball|
|    ABALDE Tamara| Spain|         Basketball|
+-----------------+------+-------------------+
only showing top 5 rows



In [0]:
coaches.show(5)

+---------------+-------------+----------+-----+
|           Name|          NOC|Discipline|Event|
+---------------+-------------+----------+-----+
|ABDELMAGID Wael|        Egypt|  Football| NULL|
|      ABE Junya|        Japan|Volleyball| NULL|
|  ABE Katsuhiko|        Japan|Basketball| NULL|
|   ADAMA Cherif|Côte d'Ivoire|  Football| NULL|
|     AGEBA Yuya|        Japan|Volleyball| NULL|
+---------------+-------------+----------+-----+
only showing top 5 rows



In [0]:
entriesgender.show(5)

+-------------------+------+----+-----+
|         Discipline|Female|Male|Total|
+-------------------+------+----+-----+
|     3x3 Basketball|    32|  32|   64|
|            Archery|    64|  64|  128|
|Artistic Gymnastics|    98|  98|  196|
|  Artistic Swimming|   105|   0|  105|
|          Athletics|   969|1072| 2041|
+-------------------+------+----+-----+
only showing top 5 rows



In [0]:
medals.show(5)

+----+--------------------+----+------+------+-----+-------------+
|Rank|            Team/NOC|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
+----+--------------------+----+------+------+-----+-------------+
only showing top 5 rows



In [0]:
teams.show(5)

+-------+--------------+--------------------+-----+
|   Name|    Discipline|                 NOC|Event|
+-------+--------------+--------------------+-----+
|Belgium|3x3 Basketball|             Belgium|  Men|
|  China|3x3 Basketball|People's Republic...|  Men|
|  China|3x3 Basketball|People's Republic...|Women|
| France|3x3 Basketball|              France|Women|
|  Italy|3x3 Basketball|               Italy|Women|
+-------+--------------+--------------------+-----+
only showing top 5 rows



In [0]:
athletes.printSchema()
entriesgender.printSchema()
medals.printSchema()
teams.printSchema()
coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Discipline: string (nullable = true)

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)

root
 |-- Rank: integer (nullable = true)
 |-- Team/NOC: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)

root
 |-- Name: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Event: string (nullable = true)

root
 |-- Name: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



Basic Transformation

In [0]:
# top countries with the highest number of gold medals
top_gold_medals_countries = medals.orderBy("Gold", ascending=False).select("Team/NOC","Gold").show()

+--------------------+----+
|            Team/NOC|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|              France|  10|
|             Germany|  10|
|               Italy|  10|
|                Cuba|   7|
|         New Zealand|   7|
|              Brazil|   7|
|              Canada|   7|
|             Hungary|   6|
|   Republic of Korea|   6|
|               Kenya|   4|
|              Poland|   4|
|      Czech Republic|   4|
|              Norway|   4|
+--------------------+----+
only showing top 20 rows



In [0]:
# analyse the total number of participants by gender in each discipline
gender_participation = entriesgender.withColumn("Gender_Ratio", 
                        (entriesgender["Female"] / entriesgender["Total"]) * 100)
gender_participation.show(5)


+-------------------+------+----+-----+-----------------+
|         Discipline|Female|Male|Total|     Gender_Ratio|
+-------------------+------+----+-----+-----------------+
|     3x3 Basketball|    32|  32|   64|             50.0|
|            Archery|    64|  64|  128|             50.0|
|Artistic Gymnastics|    98|  98|  196|             50.0|
|  Artistic Swimming|   105|   0|  105|            100.0|
|          Athletics|   969|1072| 2041|47.47672709456149|
+-------------------+------+----+-----+-----------------+
only showing top 5 rows



In [0]:
# analyze teams and their disciplines
team_participation = teams.groupBy("Discipline", "NOC").count()
team_participation.show(5)


+-------------------+-------------+-----+
|         Discipline|          NOC|count|
+-------------------+-------------+-----+
|Artistic Gymnastics|        Italy|    1|
|      Cycling Track|  Switzerland|    2|
|Artistic Gymnastics|Great Britain|    2|
|         Water Polo|          ROC|    1|
|            Archery|      Ukraine|    2|
+-------------------+-------------+-----+
only showing top 5 rows



In [0]:
# analyze the most active coaches based on the number of events:
top_coaches = coaches.groupBy("Name").count().orderBy(desc("count")).limit(10)
top_coaches.show()


+--------------------+-----+
|                Name|count|
+--------------------+-----+
|            WANG Jie|    2|
|   FARINELLI Roberta|    2|
|       FUJIKI Mayuko|    2|
|       SZAUDER Gabor|    2|
|REQUENA PERICAS J...|    2|
|GIALLOMBARDO Patr...|    2|
|    MONTICO Loredana|    2|
|    GUERRERO Rolando|    2|
|       ELAFANDI Nour|    2|
|   CHEPAK Anastasiya|    2|
+--------------------+-----+



In [0]:
athletes.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/olympicdata/transform-data/athletes")
coaches.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/olympicdata/transform-data/coaches")
entriesgender.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/olympicdata/transform-data/entriesgender")
medals.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/olympicdata/transform-data/medals")
teams.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/olympicdata/transform-data/teams")